## GO Analysis

To examine whether the dataset contains any enriched functional terms a Gene Ontology (GO) enrichment analysis is perforemd. For the Gene Ontology (GO) enrichment analysis, we use the API of [a GO tool](https://agotool.org/). This tool is especially tailored for mass spectrometry datasets, as it takes the abundance bias in mass spectrometrey data into account ([Schölz et al., 2015](https://doi.org/10.1038/nmeth.362)).

**Requirements**:
The minimal requirement for a GO analysis is a list of proteins (UniProtIDs). This can either be
    
- a list of upregulated Proteins or
- a list of proteins with a specific PTMs

If a evidence file (in case of MaxQuant) is provided, AlphaStats will extract proteins with PTMs by default.

## Create DataSet

In [46]:
import alphastats

In [132]:
maxquant_data = alphastats.MaxQuantLoader(file="../testfiles/maxquant_proteinGroups.txt")
ds = alphastats.DataSet(
    loader = maxquant_data, 
    metadata_path = "../testfiles/maxquant_metadata.xlsx",
    sample_column = "sample" # specify the column that corresponds to the sample names in proteinGroups
)

## Get enriched proteins

In [133]:
ttest_result = ds.calculate_ttest_fc(column = "disease", group1 = "liver cirrhosis", group2 = "type 2 diabetes mellitus")

In [134]:
ttest_result = ttest_result.dropna()
enriched_proteins = ttest_result[(ttest_result["pvalue"] < 0.05) & (ttest_result["foldchange_log2"] > 0)]["Protein ID"].to_list()

In [36]:
import requests
import pandas as pd
from io import StringIO

class enrichement_df(pd.DataFrame):
    # this is that added methods dont get lost when operatons on pd Dataframe get performed
    @property
    def _constructor(self):
        return enrichement_df

    def plot_goterm(self):
        return px.scatter(self, 
            x = "FDR",
            y = "effect_size", 
            size = self["foreground_n"])
    
def go_genome(tax_id=9606, method="ptm", sample = None, protein_list = None):
        
        protein_list = "%0d".join(protein_list)
        url = r"https://agotool.org/api_orig"
        
        result = requests.post(url,
                   params={"output_format": "tsv",
                           "enrichment_method": "genome",
                           "taxid": tax_id},
                   data={"foreground": protein_list})

        result_df = enrichement_df(pd.read_csv(StringIO(result.text), sep='\t')) 
        return result_df

In [141]:
enriched_filtered = []
for e in enriched_proteins:
    if ";" not in e:
        enriched_filtered.append(e)
    else:
        enriched_filtered.append(e.split(";")[0])
    

In [181]:
ds.preprocess(normalization="zscore")

In [143]:
go_genome(protein_list = enriched_proteins).shape

(28, 20)

In [146]:
len(enriched_proteins)

53

In [147]:
len(enriched_filtered)

53

In [175]:
ds.metadata #1_31_C6
bg_protein = "%0d".join(ds.mat.loc["1_31_C6"].dropna().index.to_list())
intensity = [str(intensity) for intensity in ds.mat.loc["1_31_C6"].dropna().values.tolist()]
bg_intensity = "%0d".join(intensity)

In [178]:
url = r"https://agotool.org/api_orig"
result = requests.post(url,
                   params={"output_format": "tsv",
                           "enrichment_method": "abundance_correction"},
                   data={"foreground": e_p,
                         "background": bg_protein,
                         "background_intensity": bg_intensity})
result_df = enrichement_df(pd.read_csv(StringIO(result.text), sep='\t')) 

In [154]:
ds.mat.loc["1_31_C6"].values.tolist()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 146060000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4022700000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6373900000.0,
 0.0,
 0.0,
 0.0,
 34715000.0,
 0.0,
 0.0,
 15423000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3974800000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 13129000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 395050000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 250190000000.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.0,
 0.0,
 19510000.0,
 0.0,
 0.0,
 0.0,
 89677000.0,
 0.0,
 62792000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5271400.0

In [144]:
go_genome(protein_list = enriched_filtered).shape

(89, 20)

In [118]:
go_genome(protein_list = enriched_proteins)

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,foreground_count,foreground_n,background_count,background_n,foreground_ids,s_value,rank,funcEnum,category,etype
0,GOCC:0043227,4,Membrane-bounded organelle,-1,u,1.290577e-06,0.002781,-0.372970,0.043478,0.416448,2,46,8578,20598,Q6EMK4;P04275,-2.196501,1,1850,Gene Ontology cellular component TEXTMINING,-20
1,GO:0009987,2,cellular process,-1,u,1.232634e-06,0.009163,-0.575740,0.108696,0.684435,5,46,14098,20598,O43866;Q9BWP8;P11226;Q6EMK4;P04275,-3.402142,1,7114,Gene Ontology biological process,-21
2,GO:0050789,3,regulation of biological process,-1,u,6.817862e-07,0.009163,-0.430968,0.108696,0.539664,5,46,11116,20598,O43866;Q9BWP8;P11226;Q6EMK4;P04275,-2.657503,2,14771,Gene Ontology biological process,-21
3,GO:0050794,4,regulation of cellular process,-1,u,7.555297e-07,0.009163,-0.402665,0.108696,0.511360,5,46,10533,20598,O43866;Q9BWP8;P11226;Q6EMK4;P04275,-2.465012,3,14775,Gene Ontology biological process,-21
4,GO:0008152,2,metabolic process,-1,u,7.347649e-06,0.024495,-0.294089,0.065217,0.359307,3,46,7401,20598,O43866;Q9BWP8;P11226,-1.509811,4,6480,Gene Ontology biological process,-21
5,GO:0110165,2,cellular anatomical entity,-1,u,3.461864e-07,0.000704,-0.734670,0.173913,0.908583,8,46,18715,20598,O43866;Q9BWP8;P0DOX6;P11226;Q8NF17;S6AWD6;Q6EM...,-4.746477,1,26509,Gene Ontology cellular component,-22
6,GO:0005622,2,intracellular,-1,u,1.089404e-06,0.001070,-0.654853,0.065217,0.720070,3,46,14832,20598,O43866;Q6EMK4;P04275,-3.904762,2,24128,Gene Ontology cellular component,-22
7,GO:0043231,5,intracellular membrane-bounded organelle,-1,u,1.282567e-06,0.001070,-0.541773,0.043478,0.585251,2,46,12055,20598,Q6EMK4;P04275,-3.192082,3,25517,Gene Ontology cellular component,-22
8,GO:0005737,3,cytoplasm,-1,u,1.233369e-06,0.001070,-0.515567,0.065217,0.580785,3,46,11963,20598,O43866;Q6EMK4;P04275,-3.046438,4,24190,Gene Ontology cellular component,-22
9,GO:0016020,3,membrane,-1,u,9.076642e-06,0.002971,-0.318551,0.152174,0.470725,7,46,9696,20598,O43866;Q9BWP8;P0DOX6;P11226;Q8NF17;S6AWD6;Q6EMK4,-1.606160,5,24556,Gene Ontology cellular component,-22


In [119]:
e_p = []
for e in enriched_proteins:
    e = e.split(";")
    e_p.append(e)

In [120]:
flat_list = [item for sublist in e_p
             for item in sublist]

In [121]:
genome_result = go_genome(protein_list = flat_list)

In [123]:
genome_result[(genome_result.term.str.contains("GO:")) & (genome_result.over_under.str.contains("o"))]

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,foreground_count,foreground_n,background_count,background_n,foreground_ids,s_value,rank,funcEnum,category,etype
43,GO:0002250,4,adaptive immune response,-1,o,4.285169e-07,0.003983,0.120922,0.152381,0.031459,32,210,648,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;P02745;P02746...,0.770033,31,4565,Gene Ontology biological process,-21
57,GO:0006956,8,complement activation,-1,o,8.041116e-07,0.003983,0.102776,0.109524,0.006748,23,210,139,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.626385,45,5968,Gene Ontology biological process,-21
58,GO:0006959,4,humoral immune response,-1,o,9.049452e-07,0.003983,0.102687,0.119048,0.016361,25,210,337,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.620575,46,5971,Gene Ontology biological process,-21
60,GO:0002253,7,activation of immune response,-1,o,2.654303e-07,0.003983,0.092532,0.109524,0.016992,23,210,350,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.608494,48,4568,Gene Ontology biological process,-21
62,GO:0006955,3,immune response,-1,o,2.800732e-06,0.003983,0.104440,0.180952,0.076512,38,210,1576,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.579928,50,5967,Gene Ontology biological process,-21
67,GO:0006958,9,"complement activation, classical pathway",-1,o,1.991160e-07,0.003983,0.079743,0.085714,0.005971,18,210,123,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;P02745;P02746...,0.534348,55,5970,Gene Ontology biological process,-21
69,GO:0050778,6,positive regulation of immune response,-1,o,8.582833e-07,0.003983,0.086225,0.114286,0.028061,24,210,578,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.523071,57,14768,Gene Ontology biological process,-21
71,GO:0007155,3,cell adhesion,-1,o,1.512802e-06,0.003983,0.088038,0.133333,0.045296,28,210,933,20598,A0A024R7C1;A0A0S2Z3Y1;A0A140GPP7;A0A172Q3A0;B2...,0.512398,59,6090,Gene Ontology biological process,-21
73,GO:0002376,2,immune system process,-1,o,2.675536e-05,0.005406,0.101119,0.214286,0.113166,45,210,2331,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.462377,61,4644,Gene Ontology biological process,-21
77,GO:0050776,5,regulation of immune response,-1,o,9.860239e-06,0.003983,0.074626,0.119048,0.044422,25,210,915,20598,A0A024RAB9;A0A024RAG6;A0A0A0MSV6;A6XNE2;P02745...,0.373585,65,14766,Gene Ontology biological process,-21


In [110]:
loader = alphastats.DIANNLoader(file="../testfiles/diann_report_final.pg_matrix.tsv")
metadata_path = "../testfiles/diann_metadata.xlsx"
obj = alphastats.DataSet(
            loader=loader,
            metadata_path=metadata_path,
            sample_column="analytical_sample external_id",
        )
obj.preprocess(imputation = "mean")

In [111]:
ttest_result = obj.calculate_ttest_fc(column = "grouping1", group1 = "Healthy", group2 = "Disease")
ttest_result = ttest_result.dropna()
enriched_proteins = ttest_result[(ttest_result["pvalue"] < 0.05) & (ttest_result["foldchange_log2"] > 0)]["Protein ID"].to_list()

In [114]:
ttest_result

,Protein ID,pvalue,foldchange,foldchange_log2
0,A0A024R4E5;C9J5E5;C9JBS3;C9JEJ8;C9JES8;C9JHN6;...,0.54,0.868195,-2.039093e-01
1,A0A024R6I7;A0A0G2JRN3,0.89,1.000000,-1.601713e-16
2,A0A024RBG1;O95989;Q9NZJ9,0.84,1.019841,2.834380e-02
3,A0A024RBG1;Q9NZJ9,0.82,0.979758,-2.950254e-02
4,A0A075B6H7,0.37,1.321458,4.021301e-01
5,A0A075B6I0,1.00,1.000000,0.000000e+00
6,A0A075B6I9,1.00,1.000000,-3.203427e-16
